<a href="https://colab.research.google.com/github/Caped-Crusader624/Image-processing-line-follower/blob/main/DCE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
from PIL import Image
from random import shuffle
import os

In [ ]:
import cv2
from torch.utils.data import DataLoader, Dataset
import torchvision.transforms as T
import torch
from torchvision.utils import make_grid
from torchvision.utils import save_image
import matplotlib.pyplot as plt
import random
%matplotlib inline
import matplotlib.pyplot as plt
from torchvision.utils import make_grid
from torchvision.utils import save_image
from torch.utils.data import Dataset
import torch

import torch.nn as nn
import torchvision
import torch.backends.cudnn as cudnn
import torch.optim
import sys
import argparse
import time
from torchvision import transforms


import torch.nn.functional as F
import math

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
#!unzip "/content/gdrive/My Drive/LOL.zip" -d "/content/gdrive/My Drive/lol"

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import math
#import pytorch_colors as colors
import numpy as np

In [ ]:
transform = transforms.Compose([transforms.ToPILImage(),transforms.Resize((512,512)),transforms.ToTensor()])
batch_s = 16
shuffle_dataset = True
random_seed= 9
filepath="/content/gdrive/My Drive/lol/our485/low/"
pretrain_dir="/content/gdrive/My Drive/snapshots/Epoch992.pth"
load_pretrain=False
snapshots_folder="/content/gdrive/My Drive/snapshots/"
snapshot_iter=10
display_iter=10
num_workers=4
val_batch_size=4
train_batch_size=8
num_epochs=200
grad_clip_norm=0.1
weight_decay=0.0001
lr=1e-4
lowlight_images_path=filepath
graph=[]

'''address=[]
new=[]
for file in os.listdir(filepath):
    address.append(str(os.path.join(filepath,file)))
    new.append(str(os.path.join(filepath,file[:-3]+".jpg")))

for image in range(len(address)):
    img_png = Image.open(address[image])
  
#The image object is used to save the image in jpg format
    img_png.save(new[image])'''

class ImageDataset(Dataset):
    def __init__(self):
      self.transform=transform
      self.img_folder=filepath
      self.address=[]
        
        #for folder in os.listdir(self.img_folder):
         #   self.folder_path=os.path.join(self.img_folder,folder)
          #  for file in os.listdir(self.folder_path):
           #     self.address.append(str(os.path.join(self.folder_path,file)))
    
      for file in os.listdir(self.img_folder):
        if ".jpg" in file:
          self.address.append(str(os.path.join(self.img_folder,file)))
      

    def __len__(self):
        return len(self.address)
 
    def __getitem__(self,index):

        image=cv2.imread(self.address[index])#[:,:,-1]
        #image=data_lowlight.resize((512,512), Image.ANTIALIAS)
        image=self.transform(image)
        
        #image=torch.div(image,255.0)

        

        #image = (np.asarray(image)/255.0) 
		    #image = torch.from_numpy(image).float()


        return image
        
class L_TV(nn.Module):
    def __init__(self,TVLoss_weight=1):
        super(L_TV,self).__init__()
        self.TVLoss_weight = TVLoss_weight

    def forward(self,x):
        batch_size = x.size()[0]
        h = x.size()[2]
        w = x.size()[3]
        count_h =  (x.size()[2]-1) * x.size()[3]
        count_w = x.size()[2] * (x.size()[3] - 1)
        h_t = torch.pow((x[:,:,1:,:]-x[:,:,:h-1,:]),2).sum()
        w_t = torch.pow((x[:,:,:,1:]-x[:,:,:,:w-1]),2).sum()
        return self.TVLoss_weight*2*(h_t/count_h+w_t/count_w)/batch_size
        
class L_spa(nn.Module):
    def __init__(self):
        super().__init__()
        self.left=torch.tensor([[[[0,0,0],[-1,1,0],[0,0,0]]]]).float().cuda()
        self.right=torch.tensor([[[[0,0,0],[0,1,-1],[0,0,0]]]]).float().cuda()
        self.up=torch.tensor([[[[0,-1,0],[0,1,0],[0,0,0]]]]).float().cuda()
        self.down=torch.tensor([[[[0,0,0],[0,1,0],[0,-1,0]]]]).float().cuda()
        
        self.left_w = nn.Parameter(data=self.left, requires_grad=False)
        self.right_w= nn.Parameter(data=self.right, requires_grad=False)
        self.up_w = nn.Parameter(data=self.up, requires_grad=False)
        self.down_w= nn.Parameter(data=self.down, requires_grad=False)
                
        self.n=torch.nn.AvgPool2d(4)

    def forward(self, x, y):#spatial loss
        
        x = torch.mean(x,1,keepdim=True)
        y = torch.mean(y,1,keepdim=True)
        
        x = self.n(x)
        y = self.n(y)
        

        orginal_left= nn.functional.conv2d(x , self.left_w, padding=[1],stride=[1])
        orginal_right= nn.functional.conv2d(x , self.right_w, padding=1,stride=[1])
        orginal_down= nn.functional.conv2d(x , self.down_w, padding=1,stride=[1])
        orginal_up= nn.functional.conv2d(x , self.up_w, padding=1,stride=[1])
    
        enh_left= nn.functional.conv2d(y , self.left_w, padding=1,stride=[1])
        enh_right= nn.functional.conv2d(y , self.right_w, padding=1,stride=[1])
        enh_down= nn.functional.conv2d(y , self.down_w, padding=1,stride=[1])
        enh_up= nn.functional.conv2d(y , self.up_w, padding=1,stride=[1])
    
        l_left=(torch.pow(orginal_left-enh_left,2))
        l_right=(torch.pow(orginal_right-enh_right,2))
        l_down=(torch.pow(orginal_down-enh_down,2))
        l_up=(torch.pow(orginal_up-enh_up,2))
    
        l_spa= torch.mean(l_left+l_right+l_down+l_up)

        return l_spa
    
class L_exp(nn.Module):

    def __init__(self):
        super().__init__()    
        self.pool = nn.AvgPool2d(16)
        self.mean=0.6
    def forward(self,y):
        y = torch.mean(y,1,keepdim=True)
        y=self.pool(y)
        
        d = torch.mean(torch.pow(torch.pow(y- torch.FloatTensor([self.mean] ).cuda(),2),0.5))
        
        return d
    
class L_color(nn.Module):
    def __init__(self):
        super(L_color, self).__init__()
    def forward(self, y):
        r,g,b = torch.split(y , 1, dim=1)
        Drg = torch.pow(r-g,2)
        Drb = torch.pow(r-b,2)
        Dgb = torch.pow(b-g,2)
        k = torch.mean(Drg + Drb + Dgb)
        return k  

class enhance_net_nopool(nn.Module):

	def __init__(self):
		super(enhance_net_nopool, self).__init__()

		self.relu = nn.ReLU(inplace=True)

		number_f = 32
		self.e_conv1 = nn.Conv2d(3,number_f,3,1,1,bias=True) 
		self.e_conv2 = nn.Conv2d(number_f,number_f,3,1,1,bias=True) 
		self.e_conv3 = nn.Conv2d(number_f,number_f,3,1,1,bias=True) 
		self.e_conv4 = nn.Conv2d(number_f,number_f,3,1,1,bias=True) 
		self.e_conv5 = nn.Conv2d(number_f*2,number_f,3,1,1,bias=True) 
		self.e_conv6 = nn.Conv2d(number_f*2,number_f,3,1,1,bias=True) 
		self.e_conv7 = nn.Conv2d(number_f*2,24,3,1,1,bias=True) 

		self.maxpool = nn.MaxPool2d(2, stride=2, return_indices=False, ceil_mode=False)
		self.upsample = nn.UpsamplingBilinear2d(scale_factor=2)


		
	def forward(self, x):

		x1 = self.relu(self.e_conv1(x))
		x2 = self.relu(self.e_conv2(x1))
		x3 = self.relu(self.e_conv3(x2))
		x4 = self.relu(self.e_conv4(x3))

		x5 = self.relu(self.e_conv5(torch.cat([x3,x4],1)))
		x6 = self.relu(self.e_conv6(torch.cat([x2,x5],1)))

		x_r = torch.tanh(self.e_conv7(torch.cat([x1,x6],1)))
		r1,r2,r3,r4,r5,r6,r7,r8 = torch.split(x_r, 3, dim=1)


		x = x +r1*(torch.pow(x,2)-x)
		x = x + r2*(torch.pow(x,2)-x)
		x = x + r3*(torch.pow(x,2)-x)
		enhance_image_1 = x + r4*(torch.pow(x,2)-x)		
		x = enhance_image_1 + r5*(torch.pow(enhance_image_1,2)-enhance_image_1)		
		x = x + r6*(torch.pow(x,2)-x)	
		x = x + r7*(torch.pow(x,2)-x)
		enhance_image = x + r8*(torch.pow(x,2)-x)
		r = torch.cat([r1,r2,r3,r4,r5,r6,r7,r8],1)
		return enhance_image,r

In [ ]:
def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        m.weight.data.normal_(0.0, 0.02)
    elif classname.find('BatchNorm') != -1:
        m.weight.data.normal_(1.0, 0.02)
        m.bias.data.fill_(0)



def train():
    
    os.environ['CUDA_VISIBLE_DEVICES']='0'
    torch.cuda.empty_cache()
    DCE_net = enhance_net_nopool().cuda()

    DCE_net.apply(weights_init)
    if load_pretrain == True:
        DCE_net.load_state_dict(torch.load(pretrain_dir))


    optimizer = torch.optim.Adam(DCE_net.parameters(), lr=lr, weight_decay=weight_decay)
    
    DCE_net.train()

    L_col = L_color()
    L_sp = L_spa()

    L_ex = L_exp()
    L_Tv = L_TV()


    for epoch in range(9):
        torch.cuda.empty_cache()
        print("Epoch: "+ str(epoch))
        for iteration, img_lowlight in enumerate(train_loader):
            torch.cuda.empty_cache()

            img_lowlight = img_lowlight.cuda()
            torch.cuda.empty_cache()
            #print('l'+str(img_lowlight.size()))
            enhanced_image,r = DCE_net.forward(img_lowlight)
            
            #print(enhanced_image.size())
            
            Loss_tv = 15*L_Tv(r)

            loss_s = torch.mean(L_sp(enhanced_image, img_lowlight))

            loss_co = 0.075*torch.mean(L_col(enhanced_image))

            loss_e = torch.mean(L_ex(enhanced_image))
			
			            
            
            # best_loss
            loss =   10*(loss_s + loss_co + loss_e +Loss_tv)
            #
        
            optimizer.zero_grad()
            loss.backward()
            torch.nn.utils.clip_grad_norm_(DCE_net.parameters(),grad_clip_norm)
            optimizer.step()
                        
            graph.append(loss)

            if ((iteration+1) % display_iter) == 0:
                print("Loss at iteration", iteration+1, ":", loss.item())
            if ((iteration+1) % snapshot_iter) == 0 and not(math.isnan(loss)):
                torch.save(DCE_net.state_dict(), snapshots_folder + "Epoch#991" + str(epoch) + '.pth')
                plt.plot(range(len(graph)), graph, 'g', label='Training loss')
                plt.title('model loss')
                plt.ylabel('loss')
                plt.xlabel('epoch')
                plt.legend(['train'], loc='upper left')
                plt.show()
            
            
def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        m.weight.data.normal_(0.0, 0.02)
    elif classname.find('BatchNorm') != -1:
        m.weight.data.normal_(1.0, 0.02)
        m.bias.data.fill_(0)


In [ ]:
#stop this after epoch 0 is printed and run next cell, we need to validate using .pth not train. but dataloaders are in this cell
graph=[]
if __name__ == '__main__':
    datasets=ImageDataset()
    dataset_size=len(datasets)
    indices = list(range(dataset_size))
    
    np.random.seed(9)
    np.random.shuffle(indices)
    train_indices, val_indices = indices[0:300], indices[300:]
# Creating PT data samplers and loaders:
    train_sampler = torch.utils.data.SubsetRandomSampler(train_indices)
    valid_sampler = torch.utils.data.SubsetRandomSampler(val_indices)
    
    
    train_loader = torch.utils.data.DataLoader(datasets, batch_size=batch_s, sampler=train_sampler)
    validation_loader = torch.utils.data.DataLoader(datasets, batch_size=batch_s,sampler=valid_sampler)

    if not os.path.exists(snapshots_folder):
        os.mkdir(snapshots_folder)


    train()
    
    # summarize history for loss
    plt.plot(range(300*10), graph, 'g', label='Training loss')
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train'], loc='upper left')
    plt.show()

KeyboardInterrupt: ignored

In [ ]:
#validation testing(somethin wrong)(check this)
validation_loader = torch.utils.data.DataLoader(datasets, batch_size=1,sampler=valid_sampler)


torch.cuda.empty_cache()

DCE_net=enhance_net_nopool().cuda()
DCE_net_1 = enhance_net_nopool().cuda()

DCE_net_1.load_state_dict(torch.load('/content/gdrive/My Drive/snapshots/Epoch99.pth'))
DCE_net.load_state_dict(torch.load('/content/gdrive/My Drive/snapshots/Epoch#9903.pth'))

result_path='/content/gdrive/My Drive/result' 
original_path='/content/gdrive/My Drive/original' 
twice_path='/content/gdrive/My Drive/twice'

for iteration, img_lowlight in enumerate(validation_loader):
  img_lowlight=img_lowlight.cuda()
  current='image'+str(iteration)+'.jpg'
  #img_lowlight=img_lowlight.cuda()*torch.FloatTensor([255] ).cuda()
  enhanced_image1,_= DCE_net_1(img_lowlight)
  torchvision.utils.save_image(enhanced_image1, os.path.join(result_path,current))

  #enhanced_image=enhanced_image*torch.tensor([2]).cuda()


  enhanced_image,_= DCE_net(img_lowlight)
  torchvision.utils.save_image(enhanced_image, os.path.join(twice_path,current))

  #img_lowlight=img_lowlight.cuda()*torch.FloatTensor([255] ).cuda()

  torchvision.utils.save_image(img_lowlight, os.path.join(original_path,current))
  





KeyboardInterrupt: ignored

In [ ]:
  enhanced_image,r= DCE_net(img_lowlight.cuda())


In [ ]:
print(r)

tensor([[[[-0.0572, -0.0553, -0.0566,  ..., -0.0556, -0.0488, -0.0301],
          [-0.0835, -0.1009, -0.1189,  ..., -0.1173, -0.0990, -0.0663],
          [-0.1007, -0.1357, -0.1642,  ..., -0.1641, -0.1392, -0.0948],
          ...,
          [-0.1053, -0.1508, -0.1841,  ..., -0.1910, -0.1640, -0.1103],
          [-0.0941, -0.1427, -0.1755,  ..., -0.1830, -0.1604, -0.1076],
          [-0.0453, -0.0808, -0.1117,  ..., -0.1194, -0.1051, -0.0573]],

         [[-0.1249, -0.1531, -0.1679,  ..., -0.1605, -0.1382, -0.1054],
          [-0.1748, -0.2143, -0.2312,  ..., -0.2191, -0.1888, -0.1420],
          [-0.1908, -0.2324, -0.2529,  ..., -0.2389, -0.2038, -0.1484],
          ...,
          [-0.1910, -0.2310, -0.2503,  ..., -0.2416, -0.2052, -0.1470],
          [-0.1735, -0.2065, -0.2208,  ..., -0.2139, -0.1861, -0.1356],
          [-0.1234, -0.1475, -0.1567,  ..., -0.1564, -0.1445, -0.0998]],

         [[-0.1083, -0.1457, -0.1505,  ..., -0.1366, -0.1114, -0.0767],
          [-0.1487, -0.1877, -

In [ ]:
enhanced_image,r= DCE_net_1(img_lowlight.cuda())
print(r)

tensor([[[[-0.2202, -0.2638, -0.2847,  ..., -0.2625, -0.2375, -0.1892],
          [-0.2644, -0.2784, -0.2877,  ..., -0.2678, -0.2546, -0.2345],
          [-0.2850, -0.2879, -0.2931,  ..., -0.2743, -0.2673, -0.2611],
          ...,
          [-0.2378, -0.2392, -0.2464,  ..., -0.2676, -0.2638, -0.2621],
          [-0.2151, -0.2241, -0.2390,  ..., -0.2643, -0.2549, -0.2444],
          [-0.1787, -0.2140, -0.2381,  ..., -0.2628, -0.2443, -0.2056]],

         [[-0.2277, -0.2729, -0.2939,  ..., -0.2712, -0.2458, -0.1980],
          [-0.2738, -0.2894, -0.2989,  ..., -0.2785, -0.2650, -0.2435],
          [-0.2942, -0.2989, -0.3042,  ..., -0.2854, -0.2783, -0.2706],
          ...,
          [-0.2463, -0.2488, -0.2562,  ..., -0.2782, -0.2743, -0.2709],
          [-0.2232, -0.2334, -0.2488,  ..., -0.2747, -0.2648, -0.2526],
          [-0.1864, -0.2221, -0.2466,  ..., -0.2720, -0.2531, -0.2144]],

         [[-0.2490, -0.2838, -0.3007,  ..., -0.2815, -0.2604, -0.2230],
          [-0.2839, -0.2936, -

In [ ]:
print(img_lowlight*torch.tensor([255]).cuda())
img_lowlight=img_lowlight*torch.tensor([255]).cuda()

In [ ]:
print(enhanced_image.cuda()*img_lowlight.cuda())

In [ ]:
s_path='/content/gdrive/My Drive/path'
for iteration, img_lowlight in enumerate(train_loader):
  current='image'+str(iteration)+'.jpg'
  img_low=img_lowlight.cuda()
  img_low=img_low*torch.FloatTensor([255] ).cuda()
  torchvision.utils.save_image(img_low, os.path.join(s_path,current))